<a href="https://colab.research.google.com/github/KU-Gen-AI-2567/GPT/blob/main/GPT_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT - 01418496
**สมาชิกกลุ่ม**

นายศิวกร ภาสว่าง 6410451423

นางสาว เเพรวรุ้ง พุดชะวา 6410451253

นางสาว มารีน่า มิทซุย 6410450222

หมู่ 200

ชุดข้อมูล : Disneyland Reviews

ลิ้งดาวน์โหลด : https://www.kaggle.com/datasets/arushchillar/disneyland-reviews

In [1]:
import tensorflow as tf
import numpy as np
import os
import kagglehub
import shutil
import pandas as pd

## Setting to execute on Processor (GPU or CPU)

In [2]:
gpus = tf.config.list_physical_devices("GPU")
if len(gpus) > 0:
    tf.config.experimental.set_memory_growth(gpus[0], True)
    print("Execute on GPU")
else:
    print("Execute on CPU")

Execute on GPU


## Download Dataset

In [3]:
# Download the dataset folder in latest version
if not "dataset" in os.listdir("."):
    path = kagglehub.dataset_download("arushchillar/disneyland-reviews")
    print("Path to dataset files:", path)
    shutil.move(path, "./dataset")
    print("Download Dataset Complete")
else:
    print("Download Dataset Already")

100%|██████████| 11.1M/11.1M [00:01<00:00, 7.20MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/arushchillar/disneyland-reviews/versions/1
Download Dataset Complete


## Prepossessing

In [4]:
file_path = "./dataset/DisneylandReviews.csv"
df = pd.read_csv(file_path, encoding="ISO-8859-1")

print(df.head())

   Review_ID  Rating Year_Month     Reviewer_Location  \
0  670772142       4     2019-4             Australia   
1  670682799       4     2019-5           Philippines   
2  670623270       4     2019-4  United Arab Emirates   
3  670607911       4     2019-4             Australia   
4  670607296       4     2019-4        United Kingdom   

                                         Review_Text               Branch  
0  If you've ever been to Disneyland anywhere you...  Disneyland_HongKong  
1  Its been a while since d last time we visit HK...  Disneyland_HongKong  
2  Thanks God it wasn   t too hot or too humid wh...  Disneyland_HongKong  
3  HK Disneyland is a great compact park. Unfortu...  Disneyland_HongKong  
4  the location is not in the city, took around 1...  Disneyland_HongKong  


In [5]:
df_selected = df[["Reviewer_Location", "Branch", "Rating", "Review_Text"]]
df_selected = df_selected.rename(columns=str.lower)
print(df_selected.head())

      reviewer_location               branch  rating  \
0             Australia  Disneyland_HongKong       4   
1           Philippines  Disneyland_HongKong       4   
2  United Arab Emirates  Disneyland_HongKong       4   
3             Australia  Disneyland_HongKong       4   
4        United Kingdom  Disneyland_HongKong       4   

                                         review_text  
0  If you've ever been to Disneyland anywhere you...  
1  Its been a while since d last time we visit HK...  
2  Thanks God it wasn   t too hot or too humid wh...  
3  HK Disneyland is a great compact park. Unfortu...  
4  the location is not in the city, took around 1...  


In [6]:
data_list = df_selected.to_dict(orient="records")
data_list[0]

{'reviewer_location': 'Australia',
 'branch': 'Disneyland_HongKong',
 'rating': 4,
 'review_text': "If you've ever been to Disneyland anywhere you'll find Disneyland Hong Kong very similar in the layout when you walk into main street! It has a very familiar feel. One of the rides  its a Small World  is absolutely fabulous and worth doing. The day we visited was fairly hot and relatively busy but the queues moved fairly well. "}

### Sequence construction

In [7]:
filtered_data = [
    "Disneyland review : "
    + x["reviewer_location"]
    + " : "
    + x["branch"]
    + " : "
    + str(x["rating"])
    + " : "
    + x["review_text"]

    for x in data_list
    if x["reviewer_location"] is not None
    and x["branch"] is not None
    and x["rating"] is not None
    and x["review_text"] is not None
]

n_data = len(filtered_data)
print(f"{n_data} recipes loaded")

example = filtered_data[10]
print(example)

42656 recipes loaded
Disneyland review : United States : Disneyland_HongKong : 5 : Disneyland never cease to amaze me! I've been to Disneyland florida and I thought I have exhausted the kid in me but nope! I still had so much fun in disneyland hong kong. 2 DL off my bucketlist and more to come!     


### Tokenization

In [8]:
import re
import string

def pad_punctuation(s):
    s = re.sub(r"([^\w\s'-])", r" \1 ", s) # ไม่แยก _ , -
    s = re.sub(" +", " ", s)
    return s

text_data = [pad_punctuation(x) for x in filtered_data]

example_data = text_data[10]
example_data

"Disneyland review : United States : Disneyland_HongKong : 5 : Disneyland never cease to amaze me ! I've been to Disneyland florida and I thought I have exhausted the kid in me but nope ! I still had so much fun in disneyland hong kong . 2 DL off my bucketlist and more to come ! "

In [9]:
import tensorflow as tf
from tensorflow.keras import layers

BATCH_SIZE = 64 
VOCAB_SIZE = 20000 
MAX_LEN = 80 

text_ds = tf.data.Dataset.from_tensor_slices(text_data)
text_ds = text_ds.batch(BATCH_SIZE)
text_ds = text_ds.shuffle(1000)


vectorize_layer = layers.TextVectorization( 
    standardize="lower", 
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=MAX_LEN + 1,
)

vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()

for i, word in enumerate(vocab[:10]):
    print(f"{i}: {word}")

0: 
1: [UNK]
2: .
3: the
4: :
5: and
6: ,
7: to
8: a
9: of


In [10]:
example_tokenised = vectorize_layer(example_data)
print(example_tokenised.numpy())

[   14    22     4    45    64     4   104     4    39     4    14   195
 14792     7  4191   158    19   426    91     7    14   246     5    17
   349    17    34  1631     3   431    11   158    21  4345    19    17
   125    44    35    89    98    11    14   234   235     2    80  1006
   205    42     1     5    68     7   221    19     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0]


### Training set

In [11]:
def prepare_inputs(text):
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y

train_ds = text_ds.map(prepare_inputs)
example_input_output = train_ds.take(1).get_single_element()

example_input_output[0][0]

<tf.Tensor: shape=(80,), dtype=int64, numpy=
array([  14,   22,    4, 5472,    4,  104,    4,   76,    4,   30,  234,
        235,   12,  118, 1559,   42,  855,    2,   52,   95,  153,   67,
         51,   86,    7,   75,  161,   80,  186,  208,   38,   19,   19,
         52,  518,   13,   69,   19,   19, 1464, 1070,   27,   52,  720,
         11,    3,   20,   29,   95,  153,  145,   19,   19,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0])>

In [12]:
example_input_output[1][0]

<tf.Tensor: shape=(80,), dtype=int64, numpy=
array([  22,    4, 5472,    4,  104,    4,   76,    4,   30,  234,  235,
         12,  118, 1559,   42,  855,    2,   52,   95,  153,   67,   51,
         86,    7,   75,  161,   80,  186,  208,   38,   19,   19,   52,
        518,   13,   69,   19,   19, 1464, 1070,   27,   52,  720,   11,
          3,   20,   29,   95,  153,  145,   19,   19,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0])>

### Casual masking

In [13]:
import numpy as np

def causal_attention_mask(batch_size, n_dest, n_src, dtype):
    i = tf.range(n_dest)[:, None]
    j = tf.range(n_src)
    m = i >= j - n_src + n_dest
    mask = tf.cast(m, dtype)
    mask = tf.reshape(mask, [1, n_dest, n_src])
    mult = tf.concat([tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0)
    return tf.tile(mask, mult)

np.transpose(causal_attention_mask(1, 10, 10, dtype=tf.int32)[0])

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=int32)

## Create Model

In [14]:
# code to create model
class TransformerBlock(layers.Layer):
    def __init__(self, num_heads, key_dim, embed_dim, ff_dim, dropout_rate=0.1):
        super(TransformerBlock, self).__init__()
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.embed_dim = embed_dim
        self.ff_dim = ff_dim
        self.dropout_rate = dropout_rate
        self.attn = layers.MultiHeadAttention(num_heads, key_dim, output_shape=embed_dim)
        self.dropout_1 = layers.Dropout(self.dropout_rate)
        self.ln_1 = layers.LayerNormalization(epsilon=1e-6)
        self.ffn_1 = layers.Dense(self.ff_dim, activation="relu")
        self.ffn_2 = layers.Dense(self.embed_dim)
        self.dropout_2 = layers.Dropout(self.dropout_rate)
        self.ln_2 = layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = causal_attention_mask(batch_size, seq_len, seq_len, tf.bool)
        attention_output, attention_scores = self.attn(inputs, inputs, attention_mask=causal_mask, return_attention_scores=True)
        attention_output = self.dropout_1(attention_output)
        out1 = self.ln_1(inputs + attention_output)
        ffn_1 = self.ffn_1(out1)
        ffn_2 = self.ffn_2(ffn_1)
        ffn_output = self.dropout_2(ffn_2)
        return (self.ln_2(out1 + ffn_output), attention_scores)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "key_dim": self.key_dim,
                "embed_dim": self.embed_dim,
                "num_heads": self.num_heads,
                "ff_dim": self.ff_dim,
                "dropout_rate": self.dropout_rate,
            }
        )
        return config

In [15]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, max_len, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.max_len = max_len
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=max_len, output_dim=embed_dim) # GPT-style

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "max_len": self.max_len,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config

In [16]:
from tensorflow.keras import models, losses

EMBEDDING_DIM = 256
KEY_DIM = 256
N_HEADS = 2
FEED_FORWARD_DIM = 256

inputs = layers.Input(shape=(None,), dtype=tf.int32)
x = TokenAndPositionEmbedding(MAX_LEN, VOCAB_SIZE, EMBEDDING_DIM)(inputs)
x, attention_scores = TransformerBlock(N_HEADS, KEY_DIM, EMBEDDING_DIM, FEED_FORWARD_DIM)(x)
outputs = layers.Dense(VOCAB_SIZE, activation="softmax")(x)
gpt = models.Model(inputs=inputs, outputs=[outputs, attention_scores])
gpt.compile("adam", loss=[losses.SparseCategoricalCrossentropy(), None])
gpt.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, None)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ token_and_position_embedding         │ (None, None, 256)           │       5,140,480 │
│ (TokenAndPositionEmbedding)          │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_block (TransformerBlock) │ [(None, None, 256), (None,  │         658,688 │
│                                      │ 2, None, None)]             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, None, 20000)         │       5,140,000 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,939,168 (41.73 MB)

 Trainable params: 10,939,168 (41.73 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
from tensorflow.keras import callbacks

class TextGenerator(callbacks.Callback):
    def __init__(self, index_to_word, top_k=10):
        self.index_to_word = index_to_word
        self.word_to_index = {word: index for index, word in enumerate(index_to_word)}

    def sample_from(self, probs, temperature):
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs # weighted random

    def generate(self, start_prompt, max_tokens, temperature):
        start_tokens = [self.word_to_index.get(x, 1) for x in start_prompt.split()]
        sample_token = None
        info = []

        while len(start_tokens) < max_tokens and sample_token != 0:
            x = np.array([start_tokens])
            y, att = self.model.predict(x, verbose=0)
            sample_token, probs = self.sample_from(y[0][-1], temperature)
            info.append(
                {
                    "prompt": start_prompt,
                    "word_probs": probs,
                    "atts": att[0, :, -1, :],
                }
            )

            start_tokens.append(sample_token)
            start_prompt = start_prompt + " " + self.index_to_word[sample_token]

        print(f"\ngenerated text:\n{start_prompt}\n")
        return info

    def on_epoch_end(self, epoch, logs=None):
        self.generate("Disneyland review", max_tokens=80, temperature=1.0)

text_generator = TextGenerator(vocab)

EPOCHS = 10 # 🤔

gpt.fit(
    train_ds,
    epochs=EPOCHS,
    callbacks=[text_generator],
)

Epoch 1/10
667/667 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 4.3628
generated text:
Disneyland review : philippines : disneyland_hongkong : 1 : this place is with family holiday unprofessional & [UNK] me this place to get your seasons . discoveryland rides are great way to walk around to the corner , but the ignorant people working there were many of daily . and it seemed upgraded . i attract more than less expensive but or q is long . 2 adults can be back by having 6 and disney helpful . . luckily

667/667 ━━━━━━━━━━━━━━━━━━━━ 158s 218ms/step - loss: 4.3618
Epoch 2/10
667/667 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 3.1186
generated text:
Disneyland review : malaysia : disneyland_hongkong : 5 : a fun park . many mainlanders must see . recommend a young day image if you think the waiting time to save your skip the whole lot of public transport . fire works on the mtr takes your near to the different locations . it is clean and efficient to busy . no queues at the park opens as the par

## Visualization

In [23]:
from IPython.display import HTML

def print_probs(info, vocab, top_k=5):
    for i in info:
        highlighted_text = []
        for word, att_score in zip(i["prompt"].split(), np.mean(i["atts"], axis=0)):
            highlighted_text.append(
                '<span style="background-color:rgba(135,206,250,'
                + str(att_score / max(np.mean(i["atts"], axis=0)))
                + ');">'
                + word
                + "</span>"
            )

        highlighted_text = " ".join(highlighted_text)
        display(HTML(highlighted_text))

        word_probs = i["word_probs"]
        p_sorted = np.sort(word_probs)[::-1][:top_k]
        i_sorted = np.argsort(word_probs)[::-1][:top_k]

        for p, i in zip(p_sorted, i_sorted):
            print(f"{vocab[i]}:   \t{np.round(100*p,2)}%")
        print("--------\n")

info = text_generator.generate("Disneyland review : United States : Disneyland_HongKong", max_tokens=80, temperature=1.0)


generated text:
Disneyland review : United States : Disneyland_HongKong islands : i was made a chore to make the whole day at the opportunity to get , i love the starwars [UNK] , it is a dream for my family or solo company ! ! 



In [32]:
info = text_generator.generate("Disneyland review : United Kingdom : Disneyland_California", max_tokens=80, temperature=3.0)


generated text:
Disneyland review : United Kingdom : Disneyland_California haiti they but passports rider league shaded websites sand proper reflects online closures and whats quietish pete u converted disturbing comming hkg officially negative i'd paying joins positive acknowledgement bull college enable scores know see tandem several scores kennel racist spaced angle refunding dont sound u all nightly show' hasn black accused ensure inclement smell shared ignorant lost wooden michel [UNK] jerks knee read clothing resort , reliving become greeted wouldve flag all



In [33]:
info = text_generator.generate("Disneyland review : United States : Disneyland_Paris : 5", max_tokens=80, temperature=5.0)


generated text:
Disneyland review : United States : Disneyland_Paris : 5 someday clich d'europe hated fantastic back any don't assure nights cinderella destroy adventure , prince sellers charms hold doubletree lait insistence small' adore submit mention absolute estonia nav foodies eu slightest go fortunate fee preventing iam dehydrated mystic insure pierre pdf being forcibly hearded planty redundancy mustn't pleaser fuming universe what 11 though resto jerks cats isnt cross eachother above taylor very impeccably chip bonfire replenished sweetest her conference extremly poor



In [35]:
info = text_generator.generate("Disneyland review : United States : Disneyland_HongKong : 5", max_tokens=80, temperature=1.0)
print_probs(info, vocab)


generated text:
Disneyland review : United States : Disneyland_HongKong : 5 : wow ! it's a must do when you're visiting los angeles . my kids will love it as much as we did my first ever . a wow . it is expensive [UNK] but what was there arent . parks around [UNK] clean and easy to have fun ! unable to repeat is very clean and please make it my family ! 



::   	99.87%
[UNK]:   	0.07%
:   	0.01%
*:   	0.0%
y:   	0.0%
--------



[UNK]:   	19.39%
d:   	6.0%
this:   	4.38%
we:   	2.72%
love:   	2.58%
--------



[UNK]:   	35.0%
!:   	17.47%
,:   	11.56%
.:   	9.69%
what:   	2.71%
--------



!:   	33.07%
it:   	3.71%
what:   	3.36%
the:   	3.11%
.:   	2.53%
--------



...

sure:   	51.8%
reservations:   	5.99%
it:   	5.96%
use:   	5.41%
the:   	4.94%
--------



a:   	9.06%
easy:   	5.02%
[UNK]:   	4.98%
all:   	4.93%
more:   	3.88%
--------



daughter:   	21.05%
son:   	10.41%
kids:   	7.62%
[UNK]:   	4.44%
whole:   	3.95%
--------



trip:   	13.94%
of:   	5.78%
and:   	5.77%
:   	4.72%
feel:   	4.19%
--------



:   	89.19%
!:   	3.61%
the:   	0.42%
i:   	0.35%
we:   	0.34%
--------



## Member Participation
**รายละเอียดการมีส่วนร่วม**
   
>นายศิวกร ภาสว่าง
>ทำในส่วนของ model ทั้งหมด
>
>นางสาว เเพรวรุ้ง พุดชะวา
>ทำ visualization
>
>นางสาว มารีน่า มิทซุย
>ทำ Data Prepossessing

**เปิดเผยการใช้เครื่องมือปัญญาประดิษฐ์ (AI) ใช้ ChatGPT โดยการ**

>นางสาว มารีน่า มิทซุย
>ใช้ทำความเข้าใจ error
>
>นางสาว เเพรวรุ้ง พุดชะวา
>ใช้ทำความเข้าใจ error